In [1]:
import inspect
import glob

In [2]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [3]:
from NPC import enemy

In [4]:
print(inspect.signature(enemy));
#print(inspect.getfullargspec(enemy));

(NAME, AT, PRD, EV, PR, WEAPON, DMG_DICE_N, DMG_BONUS, COU, EXP)


## Initialization

In [5]:
# For previous saved state, use saved_encounter or save_file as filename
#filename = "encounters/Scenario_Gardes_Entree_Chateau"
#filename = "encounters/Scenario_Gardes_Patrouilles"
#filename = "encounters/Scenario_Valtordu_Chef_de_Garde"
#filename = "encounters/Scenario_Rats_Pesteux"
#filename = "encounters/Scenario_Valtordu_Chapichapos"
#filename = "encounters/Scenario_Chnafon_Gronick"
#filename = "encounters/Scenario_Chnafon_Ruflak"
filename = "Sample_Encounter_1"
#filename = "save_file"

### Reading enemies

In [6]:
# Check and prepare file for loading saved encounter
if filename in ["saved_encounter", "save_file"]:
    filename = "current_save"
    relative_path = "save_file/"
    states = glob.glob(relative_path+'saved_*')
    states = [s[len(relative_path):] for s in states]
        
    compiled_save = str(len(states)) + " (NAME, AT, PRD, EV, PR, WEAPON, DMG_DICE_N, DMG_BONUS, COU, EXP)"
    for save in states:
        with open(relative_path+save, 'r') as file:
            compiled_save += "\n" + file.readline()
    
    with open(filename, 'w') as file:
        file.write(compiled_save)

In [7]:
# List of enemies
enemies = []

with open(filename, 'r') as file:
    x = file.readline().split();
    try:
        nb_enemies = int(x[0])
    except ValueError:
        print("First line should be number of enemies followed by legend.")
    
    for i in range(nb_enemies):
        # Read enemy (1 line per enemy)
        NAME, AT, PRD, EV, PR, WEAPON, DMG_DICE_N, DMG_BONUS, COU, EXP = file.readline().split();
        # Convert specific variables to int for further use
        AT, PRD, EV, PR, DMG_DICE_N, DMG_BONUS, COU, EXP = \
            int(AT), int(PRD), int(EV), int(PR), int(DMG_DICE_N), int(DMG_BONUS), int(COU), int(EXP)
        # Create enemy instance
        myVars = vars()
        myVars[NAME] = enemy(NAME, AT, PRD, EV, PR, WEAPON, DMG_DICE_N, DMG_BONUS, COU, EXP)
        enemies.append(myVars[NAME])

In [8]:
enemy_dict = {}
for opponent in enemies:
    enemy_dict.update({opponent.NAME:opponent})
    print(opponent.info(),"\n")

Rat a 7 en attaque, 5 en parade, 15 en EV, et une protection de 1.
Armé de Teeth, qui fait des dégâts 1D6+2.
Ils ont un courage de 8.
Ils donnent 8 XP en mourant. 

Bat a 9 en attaque, 9 en parade, 12 en EV, et une protection de 2.
Armé de Wings, qui fait des dégâts 2D6+1.
Ils ont un courage de 10.
Ils donnent 10 XP en mourant. 



In [9]:
@interact
def enemy_stats(Update=(0,1,1)):
    for opponent in enemies:
        print(opponent.NAME)
        print("EV = ", opponent.EV,"PR = ",opponent.PR)

interactive(children=(IntSlider(value=0, description='Update', max=1), Output()), _dom_classes=('widget-intera…

## Fight Sequence

#### Attacking

In [10]:
# Attack
for opponent in enemies:
    opponent.attack();
    
# If counter-attacked
print("--------------------");
print("If counter-attacked:\n");
for opponent in enemies:
    opponent.counter();

print("\nEnemies: ",[(bad_guy.NAME,bad_guy.EV,bad_guy.state) for bad_guy in enemies]); # Names For Counter (if needed)

Rat fait 4 de dégâts en EV à son ennemi.
Bat rate son attaque.
--------------------
If counter-attacked:

Rat n'arrives pas à parer. Dégats à calculer.
Bat n'arrives pas à parer. Dégats à calculer.

Enemies:  [('Rat', 15, 'ALIVE'), ('Bat', 12, 'ALIVE')]


In [11]:
#actions=["Attack","Defend","Counter"]
actions=["Defend","Kill"]
@interact
def enemy_update(Hit=(0,20,1),
                Opponent=enemy_dict,
                Action=actions,
                CritHit=[False,True]):
    print(Opponent.NAME)
    print("EV = ", Opponent.EV,"PR = ",Opponent.PR)
    global hit,opponent,action,crit
    opponent=Opponent
    hit=Hit
    action=Action
    crit=CritHit

interactive(children=(IntSlider(value=10, description='Hit', max=20), Dropdown(description='Opponent', options…

In [13]:
if action=="Attack":
    opponent.attack();
    opponent.counter();
if action=="Defend" and not crit:
    opponent.defend(hit);
if action=="Defend" and crit:
    opponent.take_damage(hit);
    opponent.next_phase();
if action=="Kill":
    opponent.EV = 0
    opponent.next_phase();

Rat arrive à parer l'attaque!
Rat contre-attaque.
Rat rate son attaque.
Rat est en vie avec 6 EV.
